# Trading with Python and the Binance API

## API Preparation - Acquire API Key

Navigate to [Binance paper trading](https://testnet.binance.vision/) and login with `github`.
[Generate HMAC_SHA256 Key](https://testnet.binance.vision/key/generate) and save to [binance.properties](../.credentials/binance.properties).

In order to load your Binance credentials:
```
pip install jproperties
```

In [1]:
from jproperties import Properties

properties = Properties()

with open("../.credentials/binance.properties", "rb") as binanceProperties:
  properties.load(binanceProperties)

print(properties.get("api-key"))
print(properties.get("secret-key"))

PropertyTuple(data='T676iQh9CncaNbbkPysmYVXAgJCVXHyZvoInM6dQdhZ85jHhlM1Yb7lI6vEkQDKH', meta={})
PropertyTuple(data='TzYhaKDc8w3WASWn7mGn3XFTVhhOb0lAXpJH8iwLaQAUq4noTeF8XHOsobn0aVfZ', meta={})


## Required packages

Use the unofficial Python wrapper for the Binance exchange REST API version 3.
```
pip install python-binance
```

## Connecting to the API Server